# Autoformer for TS Classification

The Autoformer is a transformer-based model for time series forecasting, 
and its encoder part can be used for extracting features from time series data.

Here's a simplified version of the Autoformer's encoder, modified for time series classification:

In [ ]:
import torch
import torch.nn as nn
import math
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

class TemporalAggregation(nn.Module):
    def __init__(self, dim_model):
        super().__init__()
        self.proj = nn.Conv1d(dim_model, dim_model, 1)

    def forward(self, x):
        x = self.proj(x)
        return x.mean(dim=2)

class AutoformerEncoder(nn.Module):
    def __init__(self, input_dim, dim_model, n_heads, num_stacks, factor):
        super().__init__()
        self.input_dim = input_dim
        self.dim_model = dim_model
        self.n_heads = n_heads
        self.num_stacks = num_stacks
        self.factor = factor

        self.proj = nn.Conv1d(input_dim, dim_model, 1)
        self.pos_enc = nn.Parameter(torch.randn(1, dim_model, factor))

        self.stacks = nn.ModuleList(
            [AutoformerStack(dim_model, n_heads, factor) for _ in range(num_stacks)]
        )

        self.aggregation = TemporalAggregation(dim_model)
        self.classifier = nn.Linear(dim_model, num_classes)

    def forward(self, x):
        x = self.proj(x)
        x = rearrange(x, 'b c l -> b l c')
        x += self.pos_enc[:, :, : x.size(1)]

        for stack in self.stacks:
            x = stack(x)

        x = self.aggregation(x)
        x = rearrange(x, 'b l c -> b c l')
        x = self.classifier(x)

        return x

class AutoformerStack(nn.Module):
    def __init__(self, dim_model, n_heads, factor):
        super().__init__()
        self.attn = AutoCorrelation(dim_model, n_heads, factor)
        self.ff = FFN(dim_model)
        self.norm1 = nn.LayerNorm(dim_model)
        self.norm2 = nn.LayerNorm(dim_model)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

class AutoCorrelation(nn.Module):
    def __init__(self, dim_model, n_heads, factor):
        super().__init__()
        self.dim_model = dim_model
        self.n_heads = n_heads
        self.factor = factor
        self.head_dim = dim_model // n_heads

        self.qkv = nn.Conv1d(dim_model, 3 * dim_model, 1)
        self.proj = nn.Conv1d(dim_model, dim_model, 1)
        self.scale = self.head_dim ** -0.5

    def forward(self, x):
        b, l, c = x.size()
        x = self.qkv(x)
        qkv = rearrange(x, 'b l (three c) -> three b c l', three=3)
        q, k, v = map(lambda t: rearrange(t, 'b c l -> b l (h c)', h=self.n_heads), qkv)

        # Compute auto-correlation matrix
        k_trans = k.transpose(-2, -1)
        r = torch.einsum('b l h d, b l h e -> b l h d e', k, k_trans) / (l - 1)

        # Compute attention scores and probabilities
        attn = torch.einsum('b l h d, b l h d e -> b l h e', q * self.scale, r)
        attn = attn.softmax(dim=-1)

        # Compute output
        v_mean = v.mean(dim=1, keepdim=True)
        v_var = v.var(dim=1, keepdim=True, unbiased=False)
        v_agg = torch.cat([v_mean, v_var], dim=-1)
        v_agg = rearrange(v_agg, 'b 1 (h d) e -> b e (h d)')
        out = torch.einsum('b l h e, b e h d -> b l h d', attn, v_agg)
        out = rearrange(out, 'b l (h d) -> b l (h d)', h=self.n_heads)
        out = self.proj(out)
        return out

class FFN(nn.Module):
    def __init__(self, dim_model):
        super().__init__()
        self.fc1 = nn.Conv1d(dim_model, dim_model * 4, 1)
        self.fc2 = nn.Conv1d(dim_model * 4, dim_model, 1)

    def forward(self, x):
        x1 = self.fc1(x)
        x1 = x1.gelu()
        x2 = self.fc2(x1)
        return x2


In [ ]:
input_dim = 1  # Number of input features
dim_model = 512  # Model dimension
n_heads = 8  # Number of attention heads
num_stacks = 2  # Number of Autoformer stacks
factor = 50  # Auto-correlation factor
num_classes = 10  # Number of classes for classification

model = AutoformerEncoder(input_dim, dim_model, n_heads, num_stacks, factor)
